<a href="https://colab.research.google.com/github/LightPotatoDev/aid/blob/master/modelstuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn

In [2]:
from google.colab import drive
import os

# 마운트
drive.mount('/content/drive')

csv_folder = '/content/drive/MyDrive/modelStuff/input/csvs/labels'
print(os.listdir(csv_folder))

Mounted at /content/drive
['labels_강경민.csv', 'construction-safety-guidelines.csv', 'part4_sentence.csv', '건설 데이터라벨링_최민혜2.CSV', 'aid_3_데이터 추출_박비원.csv']


In [3]:
# 간단한 3개 카테고리로 시작 -> 학습용
categories = ['위험경고', '예방조치', '점검사항']

# 수동 라벨링 예시 (각 카테고리당 30-50개)
labeled_examples = [
    {'text': '교각 브라켓 해체작업 시 근로자가 추락할 위험이 있습니다.', 'category': '위험경고'},
    {'text': '거푸집은 시공 중 변형이 발생되지 않도록 강성과 정밀도를 확보하여야 한다.', 'category': '예방조치'},
    {'text': '이동식 작업대차는 정기 및 수시 점검을 통하여 이상유무를 확인하여야 한다.', 'category': '점검사항'}
]

# Flair 형식으로 변환
with open('train.txt', 'w', encoding='utf-8') as f:
    for example in labeled_examples:
        f.write(f"__label__{example['category']} {example['text']}\n")

# 결과 예시(train.txt) -> __label__ , 카테고리 , 우리 넣은  모델 훈련 용 데이터셋

# __label__위험경고 교각 브라켓 해체작업 시 근로자가 추락할 위험이 있습니다.
# __label__예방조치 거푸집은 시공 중 변형이 발생되지 않도록 강성과 정밀도를 확보하여야 한다.
# __label__점검사항 이동식 작업대차는 정기 및 수시 점검을 통하여 이상유무를 확인하여야 한다.
# __label__위험경고 크레인 작업 반경 내에 근로자의 출입을 금지시켜야 한다.
# __label__예방조치 추락위험이 있는 장소에서 콘크리트 타설 작업 시에는 안전모, 안전대 등의 보호구를 항상 착용하여야 한다.
# __label__점검사항 압출잭의 압출용량이 충분한지를 확인하고, 압출력 및 압출상태 등 압출장비에 대한 이상유무도 점검하여야 한다.

# Changelog
 - 처음 데이터프레임에서 불필요한 공백을 삭제하는 코드 추가 및 코드 간략화

In [4]:
# 이게 실제로 우리가 사용할 학습데이터들.

import pandas as pd
from sklearn.model_selection import train_test_split

# CSV 파일에서 데이터 로드
filepaths = [csv_folder+'/'+f for f in os.listdir(csv_folder)]
df = pd.concat(map(pd.read_csv, filepaths), ignore_index=True)

# NaN 값 처리 및 텍스트 정제
df = df.dropna(subset=['지침내용', '지침유형', '공사종류'])         # 필수 열에 NaN이 있는 행 제거
df['지침내용'] = df['지침내용'].str.replace('\n', ' ').str.strip()  # 줄바꿈 제거 및 공백 정리
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)       # 모든 값의 양 옆 공백 제거
df['지침유형'] = df['지침유형'].str.replace(' ', '')                # 지침유형 행의 모든 공백 제거

indices_to_drop = df[df["지침유형"] == "지침유형"].index
df = df.drop(indices_to_drop) # 또는 .isin()

print(df["지침유형"].unique())
print(df['공사종류'].unique())
print(df['지침유형'].value_counts())
print(df['공사종류'].value_counts())

pd.set_option('display.max_rows', 10)
display(df)

['위험경고' '예방조치' '점검사항']
['건축' '굴착공사' '가설구조물' '교량' '공통' '양중작업']
지침유형
예방조치    538
점검사항    393
위험경고    117
Name: count, dtype: int64
공사종류
공통       301
가설구조물    251
교량       173
굴착공사     134
건축       100
양중작업      89
Name: count, dtype: int64


,지침내용,공사종류,지침유형
0,"천장 먹놓기 작업 시에는 이동식비계 등을 사용하고, 사다리를 사용하여서는 안된다.",건축,위험경고
1,경량철골 천장틀 설치 시에는 천장 높이가 높은 경우에는 비계 등을 조립하여 작업발판...,건축,위험경고
2,철물작업 시 손의 베임 등을 방지하기 위하여 장갑을 착용하여야 한다.,건축,예방조치
3,부재 적재장소는 근로자의 통행에 불편을 주지 않고 타 작업에 간섭이 되지 않는 위치...,건축,예방조치
4,"당일 시공요구량 적정여부, 작업방법 및 순서, 타공정과의 간섭 여부 등 작업 계획의...",건축,점검사항
...,...,...,...
1044,"자재를 반입할 경우에는 설계도서와의 부합여부를 검수하여야 하며, 검수 도중 불량 자...",공통,점검사항
1045,"설계도서와 시공계획서에 준하여 시공하여야 하며, 공사 도중 지반조건이 설계도서와 상...",공통,점검사항
1046,천공장비를 이동할 경우에는 장비의 전도·전락을 방지하기 위하여 이동통 로의 안전성을...,공통,예방조치
1047,천공 중에는 비산 먼지 및 소음이 심하므로 근로자에게는 방진마스크 및 귀마개를 착용...,공통,예방조치


In [5]:
# 먼저 train과 temp로 나누기 (80% 학습, 20% 임시)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

print(train_df['지침유형'].value_counts())

# temp를 다시 dev와 test로 나누기 (각각 10%)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 각 분류 차원에 대한 별도 파일 생성
with open('type_train.txt', 'w', encoding='utf-8') as f:
    for _, row in train_df.iterrows():
        text = row['지침내용']
        type_category = row['지침유형']
        f.write(f"__label__{type_category} {text}\n")

with open('type_dev.txt', 'w', encoding='utf-8') as f:
    for _, row in dev_df.iterrows():
        text = row['지침내용']
        type_category = row['지침유형']
        f.write(f"__label__{type_category} {text}\n")

with open('type_test.txt', 'w', encoding='utf-8') as f:
    for _, row in test_df.iterrows():
        text = row['지침내용']
        type_category = row['지침유형']
        f.write(f"__label__{type_category} {text}\n")

with open('construction_train.txt', 'w', encoding='utf-8') as f:
    for _, row in train_df.iterrows():
        text = row['지침내용']
        construction_category = row['공사종류']
        f.write(f"__label__{construction_category} {text}\n")

with open('construction_dev.txt', 'w', encoding='utf-8') as f:
    for _, row in dev_df.iterrows():
        text = row['지침내용']
        type_category = row['공사종류']
        f.write(f"__label__{type_category} {text}\n")

with open('construction_test.txt', 'w', encoding='utf-8') as f:
    for _, row in test_df.iterrows():
        text = row['지침내용']
        type_category = row['공사종류']
        f.write(f"__label__{type_category} {text}\n")

print(f"처리된 데이터 수: {len(df)}")

지침유형
예방조치    433
점검사항    316
위험경고     89
Name: count, dtype: int64
처리된 데이터 수: 1048


In [33]:
!head -n 5 /content/sample_data/type_train_dev_test/type_train.txt

head: cannot open '/content/sample_data/type_train_dev_test/type_train.txt' for reading: No such file or directory


<1주차 정리>

- 머신러닝 , 딥러닝 , 지도학습 , 비지도학습 , 강화학습 기본 설명

- 안전지침 pdf들이 모여있는 폴더를 순회하면서 ,
   키 : pdf의 제목 , value: 추출한 문장들 형식의 추출결과 저장

- train.csv 를 읽어서 필요한 열의 정보들을 추출

- 안전지침만을 자동으로 추출하는 엔진 제작 과정을 설명

- 안전지침 분류 모델로 사용할 Flair 모듈에 주입할 훈련용 라벨링 데이터 제작 설명

해야할 것 : 개인별 랩탑 세팅 , 개인별 라벨링 한 데이터 수집.

PyTorch(torch)는 딥러닝 프레임워크로, Flair 라이브러리가 내부적으로 PyTorch를 기반으로 작동하기 때문에 필요합니다.

Flair의 핵심 기능들은 모두 PyTorch로 구현되어 있습니다:

모델 구조: TextClassifier 같은 Flair의 모델들은 PyTorch의 nn.Module을 상속받아 구현됩니다.

학습 과정: 모델 학습은 PyTorch의 옵티마이저와 손실 함수를 사용합니다.

GPU 가속: PyTorch를 통해 CUDA를 활용한 GPU 학습이 가능합니다.

임베딩 처리: 트랜스포머 모델을 포함한 다양한 임베딩도 PyTorch를 통해 처리됩니다.

따라서 Flair를 사용하려면 반드시 PyTorch가 설치되어 있어야 합니다. PyTorch가 없으면 Flair 라이브러리 자체가 작동하지 않습니다.

In [6]:
# 1. 필요한 패키지 설치
!pip install flair transformers torch sentencepiece kobert-transformers -q
# 넘파이 - transformer 버전 조합 때문에 고민을 했었는데 tourch 를 설치 함으로써 딥러닝이 요하는 연산을 다 커버할 수 있게 되었다.
!pip install -U huggingface_hub

# 2. 디렉토리 생성
!mkdir -p /content/sample_data/type_train_dev_test
!mkdir -p /content/sample_data/construction_train_dev_test
!mkdir -p models/type-classifier
!mkdir -p models/construction-classifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# 3. 이미 생성된 파일들을 적절한 디렉토리로 이동
!mv type_train.txt /content/sample_data/type_train_dev_test/
!mv type_dev.txt /content/sample_data/type_train_dev_test/
!mv type_test.txt /content/sample_data/type_train_dev_test/
!mv construction_train.txt /content/sample_data/construction_train_dev_test/
!mv construction_dev.txt /content/sample_data/construction_train_dev_test/
!mv construction_test.txt /content/sample_data/construction_train_dev_test/

# 4. 필요한 라이브러리 및 모듈 임포트
import torch
from flair.data import Corpus, Sentence # 텍스트 데이터 처리를 위한 Corpus 클래스
from flair.datasets import ClassificationCorpus # 분류 작업을 위한 특화된 코퍼스 클래스
from flair.embeddings import TransformerDocumentEmbeddings # 트랜스포머 기반 임베딩 (BERT 등)
from flair.models import TextClassifier # 텍스트 분류 모델
from flair.trainers import ModelTrainer # 모델 학습을 위한 트레이너

import random
import numpy as np
from sklearn.model_selection import StratifiedKFold
import tempfile
import gc

# 5. GPU 확인 # colab 설정에서 하드웨어 가속기로 cpu가 아닌 gpu를 사용하도록 설정 변경.
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

CUDA 사용 가능: True
GPU 이름: Tesla T4


type_corpus = ClassificationCorpus("/content/sample_data/type_train_dev_test",  # 데이터 디렉토리 경로
                                 train_file='type_train.txt',  # 학습 데이터 파일
                                 dev_file='type_dev.txt',  # 검증 데이터 파일
                                 test_file='type_test.txt')  # 테스트 데이터 파일:

이 코드의 역할 ->

1.파일 경로 지정

- 첫 번째 인자 ("/content/sample_data/type_train_dev_test")는 데이터 파일들이 위치한 기본 디렉토리 경로입니다 ,

- train_file, dev_file, test_file 매개변수는 각각 학습, 검증, 테스트에 사용될 데이터 파일의 이름을 지정합니다.

2.데이터 로드 및 전처리:

- 지정된 파일들에서 텍스트와 라벨을 읽어옵니다.(아까 txt 파일을 열어서 봤듯이 __label__라벨명-실제내용 이런 형태로 쭉 쓰여져 있다.)
- 텍스트와 라벨을 Flair의 내부 형식으로 변환합니다.
- 학습/검증/테스트 세트를 각각 분리하여 저장합니다.

3.라벨 처리:

- 파일에서 'label'로 시작하는 라벨을 추출하고, 기본적으로 'class' 타입의 라벨로 변환합니다.
- 이렇게 생성된 코퍼스에서 나중에 make_label_dictionary()를 호출하면 라벨 사전을 얻을 수 있습니다.


결론 : type_corpus 객체는 모델 학습, 평가, 예측 등의 과정에서 데이터를 제공하는 역할 -> 모델이 이해할 수 있도록 우리가 만든 텍스트와 라벨쌍을 그 내부 구조에 맞춰 바꿔주는 역할.



---------------------------------------------------------

type_classifier = TextClassifier(document_embeddings,
                                label_dictionary=type_corpus.make_label_dictionary(label_type= 'class'), label_type='class')
 이 코드의 역할 :

1.분류기 생성:

- TextClassifier 인스턴스를 생성합니다.

2.임베딩 모델 연결:

- document_embeddings 매개변수를 통해 텍스트를 벡터로на 변환하는 임베딩 모델을 분류기에 연결합니다.

3.라벨 사전 설정:

- type_corpus.make_label_dictionary(label_type='class')는 코퍼스에서 'class' 타입으로 지정된 모든 라벨을 수집하여 사전(dictionary) 형태로 만듭니다. 이 사전은 분류기가 어떤 클래스들을 예측할 수 있는지 정의합니다.

4.라벨 타입 지정:

- label_type='class' 매개변수는 분류기가 'class' 타입의 라벨을 예측할 것임을 지정합니다. 이는 코퍼스에서 사용한 라벨 타입과 일치해야 합니다.

- (결과적으로 분류된 그 카테고리의 타입이 class일 것을 의미함 -> 당연하다. 왜냐면 class 타입의 라벨들을 corpus에서 받아서 dictionary의 인덱스로 설정해 놓은 셈이기에.)

결론:

- 이 분류기는 입력 텍스트를 document_embeddings를 사용해 벡터로 변환한 후, 그 벡터를 바탕으로 라벨 사전에 정의된 클래스 중 하나로 분류하는 작업을 수행합니다. 'class' 타입으로 지정된 라벨들(원래 파일에서는 'label'로 시작하는 태그들)을 예측하게 됩니다.

# 랜덤 서치 + 교차 검증 방식:

하이퍼파라미터 공간 정의:

- 학습률: [0.001, 0.1] 연속 공간
- 배치 크기: [8, 16, 32, 64]
- 에폭 수: [5, 10, 20]


랜덤 조합 생성:

- 정의된 공간에서 15 ~ 20개 하이퍼 파라미터 조합을 무작위로 추출


각 랜덤 조합에 교차 검증 적용:

- 각 조합마다 5-fold 교차 검증 수행
- 각 조합별로 5개 성능 점수의 평균 계산


최적 조합 선택:

- 교차 검증 평균 성능이 가장 높은 하이퍼 파라미터 조합 선택


최종 모델링:

- 선택된 최적 조합으로 전체 훈련 데이터 사용해 최종 모델 학습


## 교차검증이란?

- 교차 검증(Cross-validation)은 모델 성능을 더 안정적으로 평가하고 과적합을 방지하는 기법입니다.
교차 검증 기본 개념:

데이터 분할:
- 전체 데이터셋을 여러 개의 부분(폴드)으로 나눕니다.
반복 학습 및 평가: 각 반복에서 하나의 폴드를 검증용으로 사용하고 나머지를 훈련용으로 사용합니다.
성능 평균화: 모든 폴드에 대한 평가 결과를 평균내어 최종 성능을 측정합니다.

가장 일반적인 형태 - k-fold 교차 검증:

예를 들어 5-fold 교차 검증에서는:

데이터를 5개 부분으로 나눕니다.
5번의 실험을 수행합니다:

1번째 실험: 1번 폴드를 검증용, 2~5번 폴드를 훈련용으로 사용

2번째 실험: 2번 폴드를 검증용, 1,3~5번 폴드를 훈련용으로 사용

...이런 식으로 5번째 실험까지 진행


5번의 실험에서 얻은 성능 지표의 평균값을 최종 성능으로 사용합니다.

교차 검증의 장점:

- 더 안정적인 성능 평가: 데이터 분할에 따른 우연의 영향을 줄여줍니다.
- 전체 데이터 활용: 모든 데이터 포인트가 훈련과 검증에 모두 사용됩니다.
- 과적합 감지: 폴드 간 성능 차이가 크면 과적합의 징후일 수 있습니다.
- 하이퍼파라미터 튜닝 신뢰성 향상: 단일 검증 세트보다 더 안정적인 평가를 제공합니다.

- 텍스트 분류 모델 튜닝에서는 5-fold나 10-fold 교차 검증을 많이 사용하며, 각 하이퍼파라미터 조합에 대해 교차 검증 점수의 평균을 비교하여 최적의 조합을 선택합니다.

### 교차검증 수행 예시(구체적)

5-fold 교차 검증의 정확한 과정:

- 데이터 분할: 전체 데이터를 5개의 동일한 크기의 폴드로 나눕니다.

  각 폴드는 전체 데이터의 20%를 포함


- 1차 반복:

  검증용: 1번 폴드(전체의 20%)
  훈련용: 2,3,4,5번 폴드(전체의 80%)
  모델을 처음부터 학습 후 1번 폴드로 검증


- 2차 반복:

  검증용: 2번 폴드(전체의 20%)
  훈련용: 1,3,4,5번 폴드(전체의 80%)
  새 모델을 처음부터 학습 후 2번 폴드로 검증


- 3~5차 반복:

  각 폴드를 한 번씩 검증용으로 사용
  항상 나머지 4개 폴드(80%)로 훈련


- 최종 결과:

  5번의 검증 성능 평균을 해당 하이퍼파라미터의 성능으로 결정



각 폴드가 검증용으로 사용될 때, 그 폴드 전체가 검증에 사용됩니다(폴드 내에서 다시 나누지 않음). 이렇게 모든 데이터가 한 번씩 검증에 사용되고, 모든 데이터 포인트가 모델 평가에 기여하게 됩니다.

#올바른 머신러닝 평가 파이프라인:

1. 최초 데이터 분리:

  전체 데이터를 훈련/검증용(80%)과 테스트용(20%)으로 분리
  테스트 데이터는 "봉인"하여 최종 평가 전까지 절대 사용하지 않음



2. 하이퍼파라미터 최적화 (랜덤 서치 + 교차 검증):

  훈련/검증용 데이터(80%)만 가지고 교차 검증 수행
  각 하이퍼파라미터 조합마다 k-fold 교차 검증으로 안정적인 평가
  평균 성능이 가장 높은 하이퍼파라미터 조합 선택



3. 최종 모델 학습:

  선택된 최적 하이퍼파라미터로
  전체 훈련/검증용 데이터(80%)를 사용해 최종 모델 학습


4.  최종 평가:

  처음에 분리해 놓은 테스트 데이터(20%)로
  최종 모델의 성능을 한 번만 평가
  이 성능이 가장 신뢰할 수 있는 일반화 성능 지표


In [8]:
def random_search_with_cross_validation(corpus, document_embeddings, n_trials=15, n_folds=2, label_type='class'): # 15 ~ 20 회가 랜덤 수행의 결과가 좋을 가능성이 높지만 , gpu시간을 고려해서 우선 15회로 설정.
    """
    랜덤 하이퍼파라미터 서치 + 교차 검증
    """
    # 데이터 준비
    sentences = corpus.train
    texts = [s.to_plain_string() for s in sentences]
    labels = [s.get_label(label_type).value for s in sentences]

    # 계층적 폴드 분할 -> labels-texts 를 균등하게(shuffle = true) fold 만큼의 카테고리로 나누기.
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    fold_indices = list(skf.split(texts, labels))

    # 하이퍼파라미터 범위 정의
    learning_rate_range = [0.005, 0.1]  # 학습률 범위
    batch_sizes = [8, 16, 32]  # 배치 크기 옵션
    epochs_range = [5, 12]  # 에폭 범위(이 범위를 넘어서는 수가 설정되면 바로 과적합 나옴.)

    # 결과 저장용
    best_score = 0
    best_params = None
    results = []

    # 랜덤 서치 시작
    for trial in range(n_trials):
        # 랜덤 하이퍼파라미터 생성
        lr = np.exp(random.uniform(np.log(learning_rate_range[0]),
                                  np.log(learning_rate_range[1])))
        batch_size = random.choice(batch_sizes)
        epochs = random.randint(epochs_range[0], epochs_range[1])

        params = {
            'learning_rate': lr,
            'batch_size': batch_size,
            'epochs': epochs
        }

        print(f"시도 {trial+1}/{n_trials}: lr={lr:.4f}, batch_size={batch_size}, epochs={epochs}")

        # 교차 검증 수행
        fold_scores = []
        valid_fold_count = 0  # 유효한 폴드 수 추적

        for fold_idx, (train_idx, val_idx) in enumerate(fold_indices): # fold_idx 는 해당 폴드가 몇번째인지 , (train_idx , val_idx) 는 해당 폴드에 지정된 np.array
            print(f"  폴드 {fold_idx+1}/{n_folds} 학습 중...")

            # 이 폴드의 훈련/검증 문장 추출
            train_sentences = [sentences[i] for i in train_idx]
            val_sentences = [sentences[i] for i in val_idx]

            # 임시 파일 및 코퍼스 생성 -> 그냥 label-texts 쌍만 우리가 만들어 놓은 상태라서 이를 학습용 파일 즉 flair이 사용할 수 있는 형태의 텍스트 임시 파일로 변형.
            with tempfile.TemporaryDirectory() as temp_dir:
                # 훈련/검증 파일 생성 -> 임시 디렉터리 생성 후 그 내에 임시 파일 생성 -> 각 순번의 훈련이 끝나면 삭제.
                train_file = os.path.join(temp_dir, "train.txt")
                dev_file = os.path.join(temp_dir, "dev.txt")

                # 훈련 파일 작성
                with open(train_file, 'w', encoding='utf-8') as f:
                    for s in train_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 검증 파일 작성
                with open(dev_file, 'w', encoding='utf-8') as f:
                    for s in val_sentences:
                        f.write(f"__label__{s.get_label(label_type).value} {s.to_plain_string()}\n")

                # 임시 코퍼스 생성
                temp_corpus = ClassificationCorpus(temp_dir, train_file="train.txt",
                                                 dev_file="dev.txt", test_file=None)

                # 분류기 생성
                classifier = TextClassifier(document_embeddings,
                                          label_dictionary=temp_corpus.make_label_dictionary(label_type),
                                          label_type=label_type)

                # 트레이너 생성 및 학습
                trainer = ModelTrainer(classifier, temp_corpus)
                try:
                    result = trainer.train(
                        base_path=os.path.join(temp_dir, f"model"),
                        learning_rate=lr,
                        mini_batch_size=batch_size,
                        max_epochs=epochs
                    )
                    # 결과 출력해보기
                    print(f"  훈련 결과: {result}")

                    # 결과 구조에 따라 적절히 접근
                    # 현재 폴드에서 학습된 모델을 로드해서 직접 평가하는 것이 더 안전
                    current_fold_model_path = os.path.join(temp_dir, f"model/best-model.pt")

                    # 현재 폴드에서 학습된 모델로 직접 점수 계산
                    current_fold_model = TextClassifier.load(current_fold_model_path)

                    # 현재 폴드의 검증 세트에서 직접 평가
                    correct_predictions = 0
                    total_samples = 0

                    for sentence in val_sentences:
                        # 문장 복제 (레이블 정보 보존)
                        test_sentence = Sentence(sentence.to_plain_string())
                        true_label = sentence.get_label(label_type).value

                        # 예측
                        current_fold_model.predict(test_sentence)
                        pred_label = test_sentence.get_label('class').value

                        if pred_label == true_label:
                            correct_predictions += 1
                        total_samples += 1

                    # 현재 폴드의 정확도 계산
                    current_fold_accuracy = correct_predictions / total_samples if total_samples > 0 else 0
                    fold_scores.append(current_fold_accuracy)
                    valid_fold_count += 1  # 유효한 폴드 증가
                    print(f"  현재 폴드 정확도: {current_fold_accuracy:.4f}")

                    # 메모리 정리
                    del current_fold_model
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    gc.collect()

                except Exception as e:
                    print(f"  오류 발생: {e}")
                    # 오류 발생 시 점수를 추가하지 않음 (유효 폴드 수가 증가하지 않음)

        # 이 조합의 평균 점수 계산
        if valid_fold_count > 0:  # 유효한 폴드가 있을 때만 처리
            avg_score = sum(fold_scores) / valid_fold_count
            print(f"  평균 성능: {avg_score:.4f} (유효한 폴드: {valid_fold_count}/{n_folds})")

            results.append({
                'params': params,
                'score': avg_score,
                'fold_scores': fold_scores,
                'valid_folds': valid_fold_count
            })

            # 최고 성능 갱신
            if avg_score > best_score:
                best_score = avg_score
                best_params = params
        else:
            print(f"  이 하이퍼파라미터 조합에 대한 유효한 결과 없음")

    # 결과 정렬 및 출력
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)

    print("\n=== 랜덤 서치 결과 요약 ===")
    for i, result in enumerate(sorted_results[:5]):  # 상위 5개만 표시
        p = result['params']
        print(f"{i+1}위: lr={p['learning_rate']:.4f}, batch_size={p['batch_size']}, epochs={p['epochs']} "
              f"- 점수: {result['score']:.4f} (유효 폴드: {result['valid_folds']}/{n_folds})")

    print(f"\n최적 하이퍼파라미터: {best_params}, 점수: {best_score:.4f}")

    return best_params, sorted_results

In [9]:
construction_corpus = ClassificationCorpus("/content/sample_data/construction_train_dev_test",  # 데이터 디렉토리 경로
                                         train_file='construction_train.txt',  # 학습 데이터 파일
                                         dev_file='construction_dev.txt',  # 검증 데이터 파일
                                         test_file='construction_test.txt', # 테스트 데이터 파일
                                         )

document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')


# 앞서 생성한 document_embeddings를 재사용 (모델 공유)
construction_classifier = TextClassifier(document_embeddings,
                                       label_dictionary=construction_corpus.make_label_dictionary(label_type= 'class'), label_type='class')  # 공사 유형 레이블 사전 생성



#랜덤서치 하이퍼파라미터 최적화 수행
best_params, results = random_search_with_cross_validation(
    corpus=construction_corpus,
    document_embeddings=document_embeddings,
    n_trials=2,  # 10번의 랜덤 시도 -> 즉!! "n-trials * (폴드수) " -> 만큼의 accuracy 측정이 있겠죠.
    n_folds=5     # 5-fold 교차 검증
)

2025-05-15 11:31:23,142 Reading data from /content/sample_data/construction_train_dev_test
2025-05-15 11:31:23,144 Train: /content/sample_data/construction_train_dev_test/construction_train.txt
2025-05-15 11:31:23,144 Dev: /content/sample_data/construction_train_dev_test/construction_dev.txt
2025-05-15 11:31:23,145 Test: /content/sample_data/construction_train_dev_test/construction_test.txt
2025-05-15 11:31:23,187 Initialized corpus /content/sample_data/construction_train_dev_test (label type name is 'class')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

...

2025-05-15 11:47:00,854 DEV : loss 0.030980654060840607 - f1-score (micro avg)  0.988
2025-05-15 11:47:00,894  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.013362978235919021]


2025-05-15 11:47:03,749 ----------------------------------------------------------------------------------------------------
2025-05-15 11:47:03,753 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  5.32it/s]

2025-05-15 11:47:06,152 
Results:
- F-score (micro) 1.0
- F-score (macro) 1.0
- Accuracy 1.0

By class:
              precision    recall  f1-score   support

          공통     1.0000    1.0000    1.0000        20
          교량     1.0000    1.0000    1.0000        14
       가설구조물     1.0000    1.0000    1.0000        14
        굴착공사     1.0000    1.0000    1.0000        12
          건축     1.0000    1.0000    1.0000         5
        양중작업     1.0000    1.0000    1.0000         2

    accuracy                         1.0000        67
   macro avg     1.0000    1.0000    1.0000        67
weighted avg     1.0000    1.0000    1.0000        67

2025-05-15 11:47:06,153 ----------------------------------------------------------------------------------------------------


  훈련 결과: {'test_score': 1.0}
  현재 폴드 정확도: 0.9880
  평균 성능: 0.9869 (유효한 폴드: 5/5)

=== 랜덤 서치 결과 요약 ===
1위: lr=0.0267, batch_size=8, epochs=9 - 점수: 0.9869 (유효 폴드: 5/5)
2위: lr=0.0083, batch_size=8, epochs=9 - 점수: 0.8749 (유효 폴드: 5/5)

최적 하이퍼파라미터: {'learning_rate': np.float64(0.026725956471838043), 'batch_size': 8, 'epochs': 9}, 점수: 0.9869


random_state=42의 기능:

- 재현성 보장: 같은 코드를 여러 번 실행해도 항상 동일한 데이터 분할 결과가 나오도록 함
무작위성 제어: 데이터를 섞는(shuffle) 과정에서 난수 생성기의 시드 값을 고정 -> shuffle = true의 기능은 random 알고리즘의 난수값 고정여부를 참으로 설정 , 그 고정 난수값 random state = 42

- 실험 비교 용이: 하이퍼파라미터만 바꿔서 실험해도 같은 데이터 분할을 사용하므로 공정한 비교 가능
값의 의미: 42라는 숫자 자체는 특별한 의미가 없으며, 관례적으로 많이 사용되는 값임

create_stratified_folds 함수의 기능:

- 계층적 데이터 분할: 각 클래스의 비율을 모든 폴드에 동일하게 유지하며 분할(우리의 훈련용 데이터를 그냥 나누기에는 비슷한 category들끼리 모여있어 어려움이 있음

 -> 훈련용 데이터 fold 5개를 각 label 별로 일정한
비율로 생성
-> 그리고 그 fold 안에서 또 훈련용/시험용 데이터 분할)

데이터 균형 유지:

- 예: 원본 데이터에 '교량공사' 30%, '터널공사' 20%, '도로공사' 50%의 비율이라면
각 폴드에도 이 비율이 유지됨


샘플링 편향 방지:
- 특정 카테고리가 특정 폴드에만 집중되는 것을 방지

교차 검증 준비:
-  각 폴드별 훈련/검증 데이터 인덱스를 생성하여 교차 검증에 사용할 수 있게 함

이 함수는 교차 검증의 품질을 높이기 위한 핵심 단계로, 특히 클래스 불균형이 있는 텍스트 분류 데이터셋에서 매우 중요합니다.

그러면 이렇게 fold하는 scikit-learn 패키지를 사용했을때 그 결과는 어떻게 나오는가?

fold_indices 안에 들어있는 데이터는 다음과 같은 형식입니다:

python[

      (train_indices_fold1, val_indices_fold1),  # 1번 폴드
      (train_indices_fold2, val_indices_fold2),  # 2번 폴드
      (train_indices_fold3, val_indices_fold3),  # 3번 폴드
      (train_indices_fold4, val_indices_fold4),  # 4번 폴드
      (train_indices_fold5, val_indices_fold5)   # 5번 폴드
]

여기서:

- 각 폴드는 튜플로 표현됩니다: (훈련 인덱스 배열, 검증 인덱스 배열)

- 각 인덱스 배열은 numpy.ndarray 타입:

    train_indices_fold1 = numpy.array([0, 1, 3, 4, 6, 7, ...])
    val_indices_fold1 = numpy.array([2, 5, 8, ...])

인덱스 비율:

- 훈련 인덱스: 전체 데이터의 약 80% (5-fold의 경우)
- 검증 인덱스: 전체 데이터의 약 20% (5-fold의 경우)


각 인덱스의 의미:

- 인덱스 값은 원본 데이터(texts와 labels 리스트)에서의 위치를 가리키는 정수
- 예: train_indices_fold1[0] = 3이면 원본 데이터의 3번째 항목



이 인덱스를 사용하여 원본 데이터에서 각 폴드별로 훈련/검증 데이터를 추출하고 모델을 학습/평가할 수 있습니다.


위와 같은 과정을 통해서 교차검증을 위한 fold를 생성하고 , 각 교차검증에 사용할 하이퍼파라미터를 랜덤서치 하는 과정을 수행, 각 fold별로 train-file , dev-file , corpus 생성 , 분류생성 , tainer 인스턴스 생성 , 각 train 의 결과 즉 accuracy 구하는 과정을 수행,

각 fold별로 , 각 하이퍼 파라미터 별로 수행했을때 가장 높은 acuuracy를 찾음으로써 하이퍼 파라미터 최적화 완료.


In [11]:
# 새로운 분류기 생성
final_classifier = TextClassifier(document_embeddings,
                                 label_dictionary=construction_corpus.make_label_dictionary(label_type='class'),
                                 label_type='class')

# 트레이너 생성
trainer = ModelTrainer(final_classifier, construction_corpus)

# 최종 모델 학습 (train+dev 데이터 모두 사용)
trainer.train('models/final_construction_model',
             learning_rate=best_params['learning_rate'],
             mini_batch_size=best_params['batch_size'],
             max_epochs=best_params['epochs'])

2025-05-15 11:48:03,166 Computing label dictionary. Progress:


0it [00:00, ?it/s]
838it [00:00, 4683.00it/s]

2025-05-15 11:48:03,352 Dictionary created for label 'class' with 6 values: 공통 (seen 246 times), 가설구조물 (seen 191 times), 교량 (seen 143 times), 굴착공사 (seen 109 times), 건축 (seen 76 times), 양중작업 (seen 73 times)
2025-05-15 11:48:03,359 ----------------------------------------------------------------------------------------------------
2025-05-15 11:48:03,361 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_fea


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-15 11:48:03,370  - mini_batch_size: "8"
2025-05-15 11:48:03,370  - max_epochs: "9"
2025-05-15 11:48:03,371  - shuffle: "True"
2025-05-15 11:48:03,372 ----------------------------------------------------------------------------------------------------
2025-05-15 11:48:03,373 Plugins:
2025-05-15 11:48:03,373  - AnnealOnPlateau | patience: '3', anneal_factor: '0.5', min_learning_rate: '0.0001'
2025-05-15 11:48:03,374 ----------------------------------------------------------------------------------------------------
2025-05-15 11:48:03,375 Final evaluation on model from best epoch (best-model.pt)
2025-05-15 11:48:03,376  - metric: "('micro avg', 'f1-score')"
2025-05-15 11:48:03,377 ----------------------------------------------------------------------------------------------------
2025-05-15 11:48:03,378 Computation:
2025-05-15 11:48:03,378  - compute on device: cuda:0
2025-05-15 11:48:03,379  - embedding storage: cpu
2025-05-15 11:48:03,381 ---------------------------------------

...

100%|██████████| 2/2 [00:00<00:00,  3.61it/s]


2025-05-15 11:49:48,490 DEV : loss 2.7369544506073 - f1-score (micro avg)  0.6762
2025-05-15 11:49:48,513  - 3 epochs without improvement
2025-05-15 11:49:49,676 ----------------------------------------------------------------------------------------------------
2025-05-15 11:49:49,679 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  2.88it/s]

2025-05-15 11:49:52,469 
Results:
- F-score (micro) 0.6381
- F-score (macro) 0.5602
- Accuracy 0.6381

By class:
              precision    recall  f1-score   support

       가설구조물     0.6970    0.7931    0.7419        29
          공통     0.7826    0.6000    0.6792        30
          교량     0.6250    0.7895    0.6977        19
        굴착공사     0.5000    0.6000    0.5455        10
          건축     0.4000    0.1667    0.2353        12
        양중작업     0.3750    0.6000    0.4615         5

    accuracy                         0.6381       105
   macro avg     0.5633    0.5915    0.5602       105
weighted avg     0.6404    0.6381    0.6260       105

2025-05-15 11:49:52,470 ----------------------------------------------------------------------------------------------------


{'test_score': 0.638095238095238}

In [13]:
# 최종 테스트 수행
test_result = trainer.tewt('models/final_construction_model/best-model.pt')
print(f"최종 테스트 성능: {test_result}") # 내가 랜덤서치 + 교차검증으로 추적한 최적의 하이퍼파라미터를 최초의 test 데이터에 적용했을때 나오는 성능평가점수

AttributeError: 'ModelTrainer' object has no attribute 'test'

#교차검증의 효과는 파라미터 최적화 보다는 실제 성능평가값을 더 정확하게 나타낼 수 있다는 점에 포커스를 맞추는건가?

- 맞습니다. 교차검증(Cross-validation)의 주요 목적은 실제 성능평가값을 더 정확하게 추정하는 데 있습니다.

##교차검증의 핵심 가치는:

- 데이터 활용 효율성: 제한된 데이터셋을 최대한 활용하여 모델의 일반화 성능을 평가합니다.
- 성능 평가의 신뢰성: 단일 테스트셋에 의존할 때 발생할 수 있는 편향을 줄이고, 여러 분할에서의 평균 성능을 측정함으로써 더 안정적인 평가 지표를 얻습니다.
- 과적합 감지: 학습 데이터와 검증 데이터 간의 성능 차이를 통해 모델의 과적합 여부를 확인할 수 있습니다.

## 단일 테스트셋에 의존하면 편향이 많이 발생하는가?

- 네, 단일 테스트셋에 의존하면 여러 종류의 편향이 발생할 수 있습니다:

- 샘플링 편향: 특정 테스트셋이 전체 데이터 분포를 충분히 대표하지 못할 수 있습니다. 예를 들어, 우연히 쉬운 샘플이나 어려운 샘플이 많이 포함될 수 있습니다.
- 데이터 분할 운(luck): 단일 분할에서는 우연히 "운 좋은" 또는 "운 나쁜" 분할이 발생할 수 있습니다. 즉, 특정 분할에서 모델이 비정상적으로 좋거나 나쁜 성능을 보일 수 있습니다.
- 과적합에 취약: 단일 테스트셋을 반복적으로 사용하면, 무의식적으로 이 특정 테스트셋에 최적화된 모델을 선택하게 되어 실제 환경에서의 일반화 성능이 떨어질 수 있습니다.
- 이상치(outlier) 영향: 작은 테스트셋에서는 소수의 이상치가 전체 성능 평가에 큰 영향을 미칠 수 있습니다.

교차검증은 이러한 문제들을 여러 번의 다른 데이터 분할을 통해 평균화함으로써 완화합니다. 예를 들어, k-fold 교차검증에서는 데이터를 k개의 부분으로 나누고 k번의 다른 학습-테스트 조합으로 모델을 평가합니다. 이렇게 하면 모든 데이터 포인트가 한 번은 테스트 세트에 포함되므로 더 안정적이고 일반화된 성능 추정치를 얻을 수 있습니다.

In [14]:
# 1. 지침 유형 분류 모델 학습
# 데이터셋 로드 및 준비
type_corpus = ClassificationCorpus("/content/sample_data/type_train_dev_test",  # 데이터 디렉토리 경로
                                 train_file='type_train.txt',  # 학습 데이터 파일
                                 dev_file='type_dev.txt',  # 검증 데이터 파일
                                 test_file='type_test.txt')  # 테스트 데이터 파일

#-> 이 밑에 label_type 이라는 다른 속성을 따로 설정을 하지 않았으니 우리의 type_train.txt 의 라벨은 기본 설정인 class 라벨의 타입으로 변환된다.

# 한국어에 특화된 KLUE BERT 모델을 문서 임베딩으로 사용
document_embeddings = TransformerDocumentEmbeddings('klue/bert-base')

# 지침 유형 분류기 모델 초기화
# label_dictionary는 학습 데이터에서 자동으로 생성됨
type_classifier = TextClassifier(document_embeddings,
                                label_dictionary=type_corpus.make_label_dictionary(label_type= 'class'), label_type='class')
# 마지막의 label_type = 'class' 의 의미는 예측 결과로 나올 값의 타입이 class 라는 것

# 모델 트레이너 초기화
type_trainer = ModelTrainer(type_classifier, type_corpus)

# 모델 학습 시작
# models/type-classifier 경로에 모델 저장
type_trainer.train('models/type-classifier',
                  learning_rate=0.02,  # 학습률 설정
                  mini_batch_size=16,  # 배치 크기 설정
                  max_epochs=10,
                   )  # 최대 에폭 수 설정

# 지침 유형 중 위험 경고의 데이터가 너무 적어서 그런지 -> accuracy 가 지나치게 낮게 나옴.(train용 데이터가 더 많이 필요할 듯.)

# 샘플 데이터 출력해보기
print("훈련 데이터 샘플:")
for i, sentence in enumerate(type_corpus.train):
    print(f"문장 {i}: {sentence.text}")
    print(f"레이블: {sentence.get_label('class')}")
    if i >= 2: break  # 처음 3개만 출력

2025-05-15 11:55:18,166 Reading data from /content/sample_data/type_train_dev_test
2025-05-15 11:55:18,167 Train: /content/sample_data/type_train_dev_test/type_train.txt
2025-05-15 11:55:18,168 Dev: /content/sample_data/type_train_dev_test/type_dev.txt
2025-05-15 11:55:18,169 Test: /content/sample_data/type_train_dev_test/type_test.txt
2025-05-15 11:55:18,209 Initialized corpus /content/sample_data/type_train_dev_test (label type name is 'class')
2025-05-15 11:55:19,293 Computing label dictionary. Progress:


0it [00:00, ?it/s]
838it [00:00, 5213.93it/s]

2025-05-15 11:55:19,460 Dictionary created for label 'class' with 3 values: 예방조치 (seen 433 times), 점검사항 (seen 316 times), 위험경고 (seen 89 times)
2025-05-15 11:55:19,468 ----------------------------------------------------------------------------------------------------
2025-05-15 11:55:19,470 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): 


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-15 11:55:20,440 epoch 1 - iter 5/53 - loss 1.62210064 - time (sec): 0.95 - samples/sec: 84.58 - lr: 0.020000 - momentum: 0.000000
2025-05-15 11:55:21,401 epoch 1 - iter 10/53 - loss 1.54063457 - time (sec): 1.91 - samples/sec: 83.93 - lr: 0.020000 - momentum: 0.000000
2025-05-15 11:55:22,320 epoch 1 - iter 15/53 - loss 1.49796633 - time (sec): 2.83 - samples/sec: 84.94 - lr: 0.020000 - momentum: 0.000000
2025-05-15 11:55:23,298 epoch 1 - iter 20/53 - loss 1.42164707 - time (sec): 3.80 - samples/sec: 84.19 - lr: 0.020000 - momentum: 0.000000
2025-05-15 11:55:24,340 epoch 1 - iter 25/53 - loss 1.40926189 - time (sec): 4.85 - samples/sec: 82.55 - lr: 0.020000 - momentum: 0.000000
2025-05-15 11:55:25,226 epoch 1 - iter 30/53 - loss 1.36832807 - time (sec): 5.73 - samples/sec: 83.75 - lr: 0.020000 - momentum: 0.000000
2025-05-15 11:55:26,355 epoch 1 - iter 35/53 - loss 1.36252960 - time (sec): 6.86 - samples/sec: 81.62 - lr: 0.020000 - momentum: 0.000000
2025-05-15 11:55:27,369 epoc

...

100%|██████████| 2/2 [00:00<00:00,  3.49it/s]

2025-05-15 11:57:17,119 DEV : loss 1.4586684703826904 - f1-score (micro avg)  0.6762


2025-05-15 11:57:17,162  - 1 epochs without improvement
2025-05-15 11:57:18,333 ----------------------------------------------------------------------------------------------------
2025-05-15 11:57:18,335 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  3.02it/s]

2025-05-15 11:57:21,235 
Results:
- F-score (micro) 0.6476
- F-score (macro) 0.5151
- Accuracy 0.6476

By class:
              precision    recall  f1-score   support

        예방조치     0.6400    0.8727    0.7385        55
        점검사항     0.6667    0.5000    0.5714        36
        위험경고     0.6667    0.1429    0.2353        14

    accuracy                         0.6476       105
   macro avg     0.6578    0.5052    0.5151       105
weighted avg     0.6527    0.6476    0.6141       105

2025-05-15 11:57:21,236 ----------------------------------------------------------------------------------------------------
훈련 데이터 샘플:
문장 0: 철거작업에 사용되는 수동 및 전동기계․기구에 대하여는 반입 전에 이상 유무 점검을 실시한다.
레이블: Sentence[13]: "철거작업에 사용되는 수동 및 전동기계․기구에 대하여는 반입 전에 이상 유무 점검을 실시한다." → 점검사항 (1.0)
문장 1: 굴착작업 진행 시 일반 슬래브는 토압, 수압 등의 횡력에 버팀 역할을 하나, 램프 슬래브는 상하층 경사로로 시공되어 구조물이 완성되기 전까지는 횡력에 대한 버팀대의 역할을 하지 못하므로 횡력에 저항토록 버팀대(Strut)등을 별도로 설치 하여 붕괴 및 변형을 방지하도록 하여야 한다.
레이블: Sentence[43]: "굴착작업 진행 시 일반 슬래브는 토압, 수압 등의 횡력에 버팀 역할을 하나

In [15]:
!pip install git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-9euhzx8_
  Running command git clone --filter=blob:none --quiet https://github.com/flairNLP/flair.git /tmp/pip-req-build-9euhzx8_
  Resolved https://github.com/flairNLP/flair.git to commit ee8596c2bbe737ec9ddeb1c6cb62fa0b161f4d84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
import flair
print(flair.__version__)

0.15.1


In [17]:
# 2. 공사 종류 분류 모델 학습
# 데이터셋 로드 및 준비
construction_corpus = ClassificationCorpus("/content/sample_data/construction_train_dev_test",  # 데이터 디렉토리 경로
                                         train_file='construction_train.txt',  # 학습 데이터 파일
                                         dev_file='construction_dev.txt',  # 검증 데이터 파일
                                         test_file='construction_test.txt', # 테스트 데이터 파일
                                         )
# test.txt에 있는 레이블 확인하기
label_counts = {}

# 테스트 데이터의 각 문장을 순회하며 레이블 카운트
for sentence in construction_corpus.train:
    label = sentence.get_label('class').value
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

# 결과 출력
print("=== 테스트 데이터(test.txt) 레이블 분포 ===")
for label, count in sorted(label_counts.items()):
    percentage = (count / sum(label_counts.values())) * 100
    print(f"레이블: {label} - {count}개 ({percentage:.2f}%)")

print(f"총 샘플 수: {sum(label_counts.values())}")
print(f"고유 레이블 수: {len(label_counts)}")
print(f"레이블 목록: {list(label_counts.keys())}")

# test.txt에 있는 레이블 확인하기
label_counts = {}

# 테스트 데이터의 각 문장을 순회하며 레이블 카운트
for sentence in construction_corpus.test:
    label = sentence.get_label('class').value
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

# 결과 출력
print("=== 테스트 데이터(test.txt) 레이블 분포 ===")
for label, count in sorted(label_counts.items()):
    percentage = (count / sum(label_counts.values())) * 100
    print(f"레이블: {label} - {count}개 ({percentage:.2f}%)")

print(f"총 샘플 수: {sum(label_counts.values())}")
print(f"고유 레이블 수: {len(label_counts)}")
print(f"레이블 목록: {list(label_counts.keys())}")

# 앞서 생성한 document_embeddings를 재사용 (모델 공유)
construction_classifier = TextClassifier(document_embeddings,
                                       label_dictionary=construction_corpus.make_label_dictionary(label_type= 'class'), label_type='class')  # 공사 유형 레이블 사전 생성

# 모델 트레이너 초기화
construction_trainer = ModelTrainer(construction_classifier, construction_corpus)

construction_trainer.train('models/construction-classifier',
                  learning_rate=0.0382,  # 학습률 설정
                  mini_batch_size=16,  # 배치 크기 설정
                  max_epochs=18
                  )  # 최대 에폭 수 설정

print("레이블 분포:", label_counts)


2025-05-15 11:57:57,535 Reading data from /content/sample_data/construction_train_dev_test
2025-05-15 11:57:57,536 Train: /content/sample_data/construction_train_dev_test/construction_train.txt
2025-05-15 11:57:57,537 Dev: /content/sample_data/construction_train_dev_test/construction_dev.txt
2025-05-15 11:57:57,538 Test: /content/sample_data/construction_train_dev_test/construction_test.txt
2025-05-15 11:57:57,577 Initialized corpus /content/sample_data/construction_train_dev_test (label type name is 'class')
=== 테스트 데이터(test.txt) 레이블 분포 ===
레이블: 가설구조물 - 191개 (22.79%)
레이블: 건축 - 76개 (9.07%)
레이블: 공통 - 246개 (29.36%)
레이블: 교량 - 143개 (17.06%)
레이블: 굴착공사 - 109개 (13.01%)
레이블: 양중작업 - 73개 (8.71%)
총 샘플 수: 838
고유 레이블 수: 6
레이블 목록: ['공통', '굴착공사', '건축', '교량', '가설구조물', '양중작업']
=== 테스트 데이터(test.txt) 레이블 분포 ===
레이블: 가설구조물 - 29개 (27.62%)
레이블: 건축 - 12개 (11.43%)
레이블: 공통 - 30개 (28.57%)
레이블: 교량 - 19개 (18.10%)
레이블: 굴착공사 - 10개 (9.52%)
레이블: 양중작업 - 5개 (4.76%)
총 샘플 수: 105
고유 레이블 수: 6
레이블 목록: ['공통', '굴착공사', '가설구조물'

0it [00:00, ?it/s]
838it [00:00, 2152.54it/s]

2025-05-15 11:57:58,151 Dictionary created for label 'class' with 6 values: 공통 (seen 246 times), 가설구조물 (seen 191 times), 교량 (seen 143 times), 굴착공사 (seen 109 times), 건축 (seen 76 times), 양중작업 (seen 73 times)
2025-05-15 11:57:58,157 ----------------------------------------------------------------------------------------------------
2025-05-15 11:57:58,160 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32001, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_fea


/usr/local/lib/python3.11/dist-packages/flair/trainers/trainer.py:545: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2025-05-15 11:57:58,174 Plugins:
2025-05-15 11:57:58,175  - AnnealOnPlateau | patience: '3', anneal_factor: '0.5', min_learning_rate: '0.0001'
2025-05-15 11:57:58,176 ----------------------------------------------------------------------------------------------------
2025-05-15 11:57:58,177 Final evaluation on model from best epoch (best-model.pt)
2025-05-15 11:57:58,178  - metric: "('micro avg', 'f1-score')"
2025-05-15 11:57:58,179 ----------------------------------------------------------------------------------------------------
2025-05-15 11:57:58,180 Computation:
2025-05-15 11:57:58,180  - compute on device: cuda:0
2025-05-15 11:57:58,181  - embedding storage: cpu
2025-05-15 11:57:58,182 ----------------------------------------------------------------------------------------------------
2025-05-15 11:57:58,182 Model training base path: "models/construction-classifier"
2025-05-15 11:57:58,183 ------------------------------------------------------------------------------------------

...

2025-05-15 12:01:28,924 DEV : loss 2.1089236736297607 - f1-score (micro avg)  0.6952
2025-05-15 12:01:28,948  - 2 epochs without improvement
2025-05-15 12:01:28,949 ----------------------------------------------------------------------------------------------------
2025-05-15 12:01:30,025 epoch 18 - iter 5/53 - loss 0.00012969 - time (sec): 1.07 - samples/sec: 74.53 - lr: 0.019100 - momentum: 0.000000
2025-05-15 12:01:30,938 epoch 18 - iter 10/53 - loss 0.00014987 - time (sec): 1.99 - samples/sec: 80.54 - lr: 0.019100 - momentum: 0.000000
2025-05-15 12:01:31,985 epoch 18 - iter 15/53 - loss 0.00023216 - time (sec): 3.03 - samples/sec: 79.11 - lr: 0.019100 - momentum: 0.000000
2025-05-15 12:01:33,064 epoch 18 - iter 20/53 - loss 0.00042484 - time (sec): 4.11 - samples/sec: 77.81 - lr: 0.019100 - momentum: 0.000000
2025-05-15 12:01:34,064 epoch 18 - iter 25/53 - loss 0.00038332 - time (sec): 5.11 - samples/sec: 78.24 - lr: 0.019100 - momentum: 0.000000
2025-05-15 12:01:34,932 epoch 18 - 

100%|██████████| 2/2 [00:00<00:00,  3.73it/s]


2025-05-15 12:01:40,065 DEV : loss 1.992823600769043 - f1-score (micro avg)  0.7048
2025-05-15 12:01:40,093  - 3 epochs without improvement
2025-05-15 12:01:41,326 ----------------------------------------------------------------------------------------------------
2025-05-15 12:01:41,329 Loading model from best epoch ...


100%|██████████| 2/2 [00:00<00:00,  2.89it/s]

2025-05-15 12:01:44,476 
Results:
- F-score (micro) 0.6667
- F-score (macro) 0.5985
- Accuracy 0.6667

By class:
              precision    recall  f1-score   support

          공통     0.7586    0.7333    0.7458        30
       가설구조물     0.9474    0.6207    0.7500        29
          교량     0.7500    0.7895    0.7692        19
        굴착공사     0.5333    0.8000    0.6400        10
          건축     0.3846    0.4167    0.4000        12
        양중작업     0.2222    0.4000    0.2857         5

    accuracy                         0.6667       105
   macro avg     0.5994    0.6267    0.5985       105
weighted avg     0.7194    0.6667    0.6797       105

2025-05-15 12:01:44,479 ----------------------------------------------------------------------------------------------------
레이블 분포: {'공통': 30, '굴착공사': 10, '가설구조물': 29, '건축': 12, '교량': 19, '양중작업': 5}


학습 단계:

- 코퍼스에 있는 텍스트 데이터들을 document_embeddings를 통해 벡터화합니다.
- 각 벡터와 해당 텍스트의 라벨('label'로 시작하는) 간의 관계를 학습합니다.
- 이 과정에서 각 클래스(라벨)에 해당하는 벡터 공간의 특성을 파악합니다.


예측 단계:

- 새로운 텍스트가 입력되면, 같은 document_embeddings를 사용해 벡터로 변환합니다.
- 이 벡터가 학습 데이터의 어떤 클래스 벡터들과 가장 유사한지 계산합니다.
- 가장 유사도가 높은 클래스를 예측 결과로 출력합니다.



이는 마치 벡터 공간에서 각 클래스별로 데이터 포인트들이 모여있는 영역(클러스터)이 형성되고, 새로운 데이터가 어떤 클러스터에 가장 가까운지를 판단하는 것과 유사합니다. 라벨 사전(label_dictionary)은 이 클래스들의 정보를 관리하는 역할을 합니다.

--------------------------------------

학습률(learning_rate=0.02):

- 모델이 학습 과정에서 가중치를 업데이트하는 스텝 크기를 결정합니다.
  값이 크면 학습이 빠르게 진행되지만 최적점을 지나칠 수 있고, 작으면 학습이 느리지만 더 정확한 최적점에 도달할 가능성이 높아집니다.



배치 크기(mini_batch_size=16):

- 한 번에 처리되는 학습 데이터 샘플의 수입니다.
메모리 사용량과 학습 안정성 사이의 균형을 맞추는 중요한 요소입니다.



최대 에폭 수(max_epochs=10):

- 전체 학습 데이터셋을 몇 번 반복해서 학습할지 결정합니다.
너무 적으면 학습이 부족할 수 있고, 너무 많으면 과적합이 발생할 수 있습니다.



이 매개변수들이 달라지면 모델 성능에 다음과 같은 영향을 미칩니다:

- 학습률 변경 시: 너무 높으면 학습이 불안정해지고, 너무 낮으면 학습이 매우 느리거나 지역 최적점에 갇힐 수 있습니다.
- 배치 크기 변경 시: 큰 배치는 더 안정적인 그래디언트를 제공하지만 메모리를 많이 사용하고 일반화 성능이 떨어질 수 있습니다. 작은 배치는 노이즈가 있지만 종종 더 나은 일반화를 제공합니다.
- 에폭 수 변경 시: 에폭이 너무 적으면 과소적합, 너무 많으면 과적합이 발생할 수 있습니다.

이러한 매개변수들은 모델의 학습 과정과 최종 성능에 직접적인 영향을 미치므로, 일반적으로 하이퍼파라미터 튜닝 과정을 통해 최적값을 찾는 것이 중요합니다.

learning-rate-visualization.svg

correct-epochs-error-graph.svg

가중치의 의미:

- 머신러닝 모델에서 가중치는 입력 데이터의 각 특성(feature)이 최종 출력에 얼마나 영향을 미치는지 결정하는 값입니다.
예를 들어 신경망에서는 뉴런 간의 연결 강도를 나타냅니다.


가중치 업데이트 과정:

- 초기에는 가중치가 랜덤한 값으로 설정됩니다.
모델이 훈련 데이터로 예측을 수행하고, 실제 값과 예측 값의 차이(오차)를 계산합니다.
이 오차를 최소화하는 방향으로 가중치를 조정합니다.
이 과정을 반복하며 모델이 점점 더 정확한 예측을 할 수 있도록 합니다.


업데이트 수식:

- 가장 기본적인 형태: 새 가중치 = 기존 가중치 - (학습률 × 오차에 대한 가중치의 영향도)
여기서 '오차에 대한 가중치의 영향도'는 경사(gradient)로 계산됩니다.
학습률(learning rate)은 한 번에 가중치를 얼마나 크게 조정할지 결정합니다.


에폭과의 관계:

- 한 에폭은 전체 훈련 데이터를 한 번 순회하는 것을 의미합니다.
각 에폭에서 모델은 훈련 데이터를 보고 가중치를 여러 번 업데이트합니다.
에폭을 반복할수록 모델의 가중치는 최적값에 가까워지게 됩니다.



이 가중치 업데이트 과정이 바로 머신러닝에서 말하는 '학습'의 본질. 모델이 데이터로부터 패턴을 학습한다는 것은 결국 이 가중치들을 데이터에 맞게 조정해나가는 과정입다.

In [22]:
# 우리가 직접 입력한

from sentence_transformers import SentenceTransformer

# 두 모델 로드
type_classifier = TextClassifier.load('/content/models/type-classifier/best-model.pt')
construction_classifier = TextClassifier.load('/content/models/construction-classifier/best-model.pt')

def classify_guideline(text):
    # 지침 유형 분류
    type_sentence = Sentence(text)
    type_classifier.predict(type_sentence)
    guideline_type = type_sentence.labels[0].value
    type_confidence = type_sentence.labels[0].score

    # 공사 종류 분류
    construction_sentence = Sentence(text)
    construction_classifier.predict(construction_sentence)
    construction_type = construction_sentence.labels[0].value
    construction_confidence = construction_sentence.labels[0].score

    return {
        'text': text,
        'guideline_type': guideline_type,
        'construction_type': construction_type,
        'type_confidence': type_confidence,
        'construction_confidence': construction_confidence
    }

NameError: name 'fcm_guidelines' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# PDF에서 추출한 모든 지침에 적용
classified_guidelines = []
for guideline in fcm_guidelines + ilm_guidelines:
    classification = classify_guideline(guideline)

    # 두 분류 모두 충분한 신뢰도를 가질 경우만 사용
    if classification['type_confidence'] > 0.7 and classification['construction_confidence'] > 0.7:
        # 임베딩 생성
        embedding = model.encode(guideline)
        classification['embedding'] = embedding
        classified_guidelines.append(classification)